In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.controller.combined_robot import *
from pkg.project_config import *

In [2]:
CONNECT_INDY = False

MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

In [3]:
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP),
    RobotConfig(1, RobotType.indy7, ((0.172,0,0.439), (0,0,0)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})]
              , connection_list=[False, CONNECT_INDY])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"kmb0": ((0,0,0), (0,0,0)), "indy1": ((0,0,0), (0,0,np.pi))}
# crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gscene.show_pose(-crob.home_pose)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
# gscene.show_pose(-crob.home_pose + ([0.2,0.2,np.pi/3,]+[0]*6))

## Set camera

In [5]:
from pkg.detector.camera.realsense import RealSense
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()
# aruco_map.print_markers()

# realsense = RealSense()
# realsense.initialize()
# camera_matrix, dist_coeffs = realsense.get_config()
# print("camera_matrix: {} \n {}".format(camera_matrix.shape, camera_matrix))
# print("dist_coeffs: {} \n {}".format(dist_coeffs.shape, dist_coeffs))

In [6]:
# img = realsense.get_image()
# pose_dict = aruco_map.get_object_pose_dict(img, camera_matrix, dist_coeffs)

## set workspace

In [7]:
gscene.set_workspace_boundary( -5, 5, -4, 4, -0.1, 1.75)

Please create a subscriber to the marker


## define robot and planning scene

In [8]:
from pkg.planning.scene import *
from demo_utils.environment import *

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.37,0.37,0.60)

MOBILE_NAME = "kmb0"
HOLD_LINK = "kmb0_platform"
HOLD_XYZ = (0.445 + 0.35, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)
HOLD_NAME = "hold0"

MOBILE_LINK = HOLD_LINK
WAYFRAME_NAME = "wayframe"

ROBOT_NAME = "indy1"
TOOL_LINK = "indy1_tcp"
TOOL_NAME = "brush_face"
TOOL_DIM = [0.15, 0.32]
TOOL_OFFSET = 0

VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
VIEW_LOC = [0]*6
HOME_POSE = np.array(VIEW_LOC + list(VIEW_POSE))
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

crob.home_pose = HOME_POSE
crob.home_dict = HOME_DICT

gtems_robot = s_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
viewpoint = add_cam(gscene, tool_link=TOOL_LINK)

pscene = PlanningScene(gscene, combined_robot=crob)
brush_face = change_tool(pscene, DummyObject(), ToolDir.down, TOOL_OFFSET, 
                         TOOL_LINK, TOOL_NAME, WayFramer, tool_dim=TOOL_DIM)

Tool Down


In [9]:
gscene.show_pose(HOME_POSE)

## planning pipeline

In [10]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)


from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.motion_filters = [GraspChecker(pscene)]

   Use a production WSGI server instead.
 * Debug mode: off


## Add action points

In [12]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [13]:
CLEARANCE = 0.001
gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link",
                   dims=(10,10,0.1), center=(0,0,-0.05-CLEARANCE), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)
floor = pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)


TABLE_DIMS = (2,0.7,0.5)
# table
gscene.create_safe(gtype=GEOTYPE.BOX, name="table", link_name="base_link",
                   dims=TABLE_DIMS, center=(2,0,0.25), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)

WP_DIMS = (0.1,0.1,0.01)
waypoints = [((-0.5,-0.25,TABLE_DIMS[2]/2),(0,0,np.pi/2)), ((+0.5,-0.25,TABLE_DIMS[2]/2),(0,0,np.pi/2)), 
             ((+0.5,0.25,TABLE_DIMS[2]/2),(0,0,-np.pi/2)), ((-0.5,0.25,TABLE_DIMS[2]/2),(0,0,-np.pi/2))]
wp_list = []
for i_w, (xyz, rpy) in enumerate(waypoints):
    wp_list.append(gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(i_w), link_name="base_link",
                                      dims=WP_DIMS, center=xyz,rpy=rpy, 
                                      color=(0.6, 0.0, 0.0, 0.5), display=True, 
                                      collision=True, fixed=True, parent="table"))
wp_task = pscene.create_subject(oname="waypoints_{}".format(i_w), gname="table", _type=WaypointTask, 
                                action_points_dict={wp.name: WayFrame(wp.name, wp, [0,0,WP_DIMS[2]/2], [0,0,0])
                                                    for wp in wp_list})



gscene.create_safe(gtype=GEOTYPE.SPHERE, name=HOLD_NAME, link_name=HOLD_LINK,
                   dims=(0.01,) * 3, center=HOLD_XYZ, rpy=HOLD_RPY, color=(1, 0, 0, 0.5), display=True,
                   collision=False,
                   fixed=True)
holder = pscene.create_binder(bname=HOLD_NAME, gname=HOLD_NAME, _type=Gripper2Tool, point=(0, 0, 0), rpy=(0, 0, 0))
holder.redundancy = {}

## Add chair

In [14]:
chair_loc = (1.5, -1, CHAIR_DIM[2]/2)
obs_all = 3
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name=CHAIR_NAME, link_name="base_link",
                   dims=CHAIR_DIM, center=chair_loc, rpy=(0,0,0),
                   color=(1,1,0,1), display=True, collision=True, fixed=False)
chair = pscene.create_subject(oname=CHAIR_NAME, gname=CHAIR_NAME, _type=CylinderObject, 
                              GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                              GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
chair.action_points_dict[CHAIR_NAME+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}

In [15]:
wayframe = gscene.create_safe(gtype=GEOTYPE.BOX, name=WAYFRAME_NAME, link_name=HOLD_LINK,
                              dims=(0.6,0.4,0.01), center=(0,0,0), rpy=(0,0,0), color=(0, 1, 0, 0.5), 
                              display=True, collision=False, fixed=True)

wayframe = PlacePoint("frame", wayframe, (0,0,0),  (0,0,0))

In [16]:
class MobileState(Enum):
    STAND=0
    MOVING=1

##
# @class MobileTask
# @brief sweep action points in alphabetical order
# @remark   state_param: boolean vector of which each element represents if each waypoint is covered or not
#           node_item: number of covered waypoints
class MobileTask(AbstractTask):
    # @param binding_pairs list of tuple (handle, attach frame) [(handle, binder), ...]
    def __init__(self, oname, geometry, wayframe, sub_binders_dict=None):
        self.oname = oname
        self.geometry = geometry
        self.wayframe = wayframe
        self.action_points_dict = {wayframe.name: wayframe}
        self.sub_binders_dict = {} if sub_binders_dict is None else sub_binders_dict
        self.binding = BindingTransform(self, None, None, None)
        self.state_param = MobileState.STAND

    ##
    # @brief (prototype) set state param
    # @param binding BindingTransform
    # @param state_param
    def set_state(self, binding, state_param):
        self.binding = deepcopy(binding)
        self.state_param = deepcopy(self.state_param)
        self.update_sub_points()

    ##
    # @brief (prototype) get updated object-level state_param component
    def get_updated_state_param(self, btf, state_param):
        print("get_updated_state_param")
        print(btf.chain)
        print(btf.T_lao)
        print(state_param)
        return deepcopy(state_param)

    ##
    # @brief (prototype) get available bindings from current binding state
    # @param from_binding current binding (subject name, handle name, actor name, actor root geometry name)
    # @param to_node_item desired node item
    # @param actor_dict
    #           dictionary of binder {binder_name: rnb-planning.src.pkg.planning.constraint.constraint_actor.Actor}
    # @param Q_dict dictionary of joint values {joint_name: value}
    # @return list of available bindings [(handle name, actor name, actor root geometry name), ...]
    def get_available_bindings(self, from_binding, to_node_item, actor_dict, Q_dict):
        ap_list = [self.wayframe]

        ctypes = [ap.ctype for ap in ap_list]
        bd_list = [actor for actor in actor_dict.values() if
                   actor.ctype in ctypes]  # bd_exclude is ignored as previous binding is re-used in sweep
        for bd in bd_list:
            self.geometry.gscene.link_control_map[bd.geometry.link_name]

        available_bindings = []
        for bd in bd_list:
            for ap in ap_list:
                if bd.check_pair(ap):
                    available_bindings.append((ap.name, bd.name, bd.geometry.name))
        if not available_bindings:
            print("=================================")
            print("=================================")
            print("=================================")
            print("Not available:{}-{}".format(self.oname, to_node_item))
            print("=================================")
            print("=================================")
            print("=================================")
        return available_bindings

    ##
    # @brief (prototype) get object-level node component
    @classmethod
    def get_node_component(cls, btf, state_param):
        return state_param.name

    ##
    # @brief (prototype) get object-level node component
    def get_neighbor_node_component_list(self, node_tem, pscene):
        if node_tem == MobileState.MOVING.name:
            return [MobileState.STAND.name]
        if node_tem == MobileState.STAND.name:
            return [MobileState.MOVING.name]
        return []

    ##
    # @brief (prototype) get all object-level node component
    def get_all_node_components(self, pscene):
        return [MobileState.STAND.name, MobileState.MOVING.name]

    ##
    # @brief make constraints. by default, empty list.
    # @remark constraint is applied when using same binding
    # @param binding_from previous binding
    # @param binding_to next binding
    def make_constraints(self, binding_from, binding_to, tol=None):
        return "Constraint not implemented yet. Use MoveitPlanner.incremental_constraint_motion=True"

In [17]:
wp_task = pscene.create_subject(oname="mobile", gname=WAYFRAME_NAME, _type=MobileTask, 
                                wayframe=wayframe)

## [Test] move chair to cover waypoints

In [18]:
initial_state = pscene.initialize_state(-crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)
goal_nodes = [('floor','STAND', 4)]
print(goal_nodes)

get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
['chair0', 'mobile', 'waypoints_3']
('floor', 'STAND', 0)
[('floor', 'STAND', 4)]


In [19]:
from pkg.planning.task.rrt_star import TaskRRTstar
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler


tplan = TaskRRTstar(pscene)
tplan.REWIND_MAX = 3
tplan.new_node_sampler = PenaltyNodeSampler(3, 3)
tplan.parent_node_sampler = UniformNodeSampler(3)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

mplan.reset_log(True)

ppline.search(initial_state, goal_nodes, verbose=True,
              timeout=0.5, timeout_loop=10, multiprocess=False,
              add_homing=3, home_pose=HOME_POSE)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_home = snode_schedule+ppline.add_return_motion(snode_schedule[-1], try_count=3, initial_state=initial_state)
ppline.play_schedule(snode_schedule_home)


get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
try: 0 - ('floor', 'STAND', 0)->('floor', 'STAND', 1)
error: 3.2
try transition motion
transition motion tried: False
Motion Plan Failure
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 0 - ('floor', 'STAND', 0)->('floor', 'STAND', 1) = fail
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
try: 0 - ('floor', 'STAND', 0)->('hold0', 'STAND', 0)
error: 3.8
try tran

get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 7 - ('floor', 'STAND', 2)->('floor', 'STAND', 3) = fail
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
try: 0 - ('floor', 'STAND', 0)->('hold0', 'STAND', 0)
Motion Filer Failure: GraspChecker
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 0 - ('floor', 'STAND', 0)->('hold0', 'STAND', 0) = fail
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile'

get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 17 - ('hold0', 'STAND', 0)->('floor', 'STAND', 0) = fail
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
try: 6 - ('floor', 'STAND', 0)->('floor', 'STAND', 0)
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 6 - ('floor', 'STAND', 0)->('floor', 'STAND', 0) = success
branching: 6->18 (4.32/10.0 s, steps/err: 2(7.99512863159 ms)/0)
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param


get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 22 - ('hold0', 'STAND', 2)->('hold0', 'STAND', 3) = fail
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
try: 13 - ('hold0', 'STAND', 1)->('hold0', 'STAND', 1)
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
result: 13 - ('hold0', 'STAND', 1)->('hold0', 'STAND', 1) = success
branching: 13->23 (8.28/10.0 s, steps/err: 2(6.75487518311 ms)/0)
get_updated_state_param
BindingChain(subject_name='mobile', handle_name='frame', actor_name='floor', actor_root_gname='floor')
MobileState.STAND
get_updated_state_par

IndexError: list index out of range

### TODO
* base movement task
* constrain push direction
* subconstraint
* skeleton

In [20]:
tplan.node_dict

{('floor', 'MOVING', 0): {('floor', 'MOVING', 1),
  ('floor', 'STAND', 0),
  ('hold0', 'MOVING', 0)},
 ('floor', 'MOVING', 1): {('floor', 'MOVING', 2),
  ('floor', 'STAND', 1),
  ('hold0', 'MOVING', 1)},
 ('floor', 'MOVING', 2): {('floor', 'MOVING', 3),
  ('floor', 'STAND', 2),
  ('hold0', 'MOVING', 2)},
 ('floor', 'MOVING', 3): {('floor', 'MOVING', 4),
  ('floor', 'STAND', 3),
  ('hold0', 'MOVING', 3)},
 ('floor', 'MOVING', 4): {('floor', 'STAND', 4), ('hold0', 'MOVING', 4)},
 ('floor', 'STAND', 0): {('floor', 'MOVING', 0),
  ('floor', 'STAND', 1),
  ('hold0', 'STAND', 0)},
 ('floor', 'STAND', 1): {('floor', 'MOVING', 1),
  ('floor', 'STAND', 2),
  ('hold0', 'STAND', 1)},
 ('floor', 'STAND', 2): {('floor', 'MOVING', 2),
  ('floor', 'STAND', 3),
  ('hold0', 'STAND', 2)},
 ('floor', 'STAND', 3): {('floor', 'MOVING', 3),
  ('floor', 'STAND', 4),
  ('hold0', 'STAND', 3)},
 ('floor', 'STAND', 4): set(),
 ('hold0', 'MOVING', 0): {('floor', 'MOVING', 0),
  ('hold0', 'MOVING', 1),
  ('hold0',